# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 15](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=15)*

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.

# The run-time defaults for all of these parameters are in NightLog.yaml
# Under Times Square, the run-time defaults always override values given here.
# Values here are used for local tests.

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-09-06"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "2"  # TODO Change to '1' to test with default before push

# Experimental Features config
exp_tally = "false"    # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    print(
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch....'
    )
    print(
        "TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP."
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
# import lsst.ts.logging_and_reporting.almanac as alm
# import lsst.ts.logging_and_reporting.reports as rep
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_sources import uniform_field_counts
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink

In [3]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [4]:
# Set default env to "usdf" and try before PUSH to repo.
summit = "https://summit-lsp.lsst.codes"
usdf = "https://usdf-rsp-dev.slac.stanford.edu"
tucson = "https://tucson-teststand.lsst.codes"

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", summit
)  # TODO try with "usdf" before push (else "summit")

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    limit=limit,
)
allrep = AllReports(allsrc=allsrc)

<head>
<style>
code {
  font-family: Consolas,"courier new";
  color: crimson;
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 105%;
}
</style>
</head>

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-09-05 to 2024-09-07 for LSSTComCamSim, LSSTComCam, LATISS

# Table of Contents
* [Night Report](#Night-Report)
    - AuxTel
    - Simonyi
* [Almanac](#almanac)
* [Summary plots of whole night](#Summary-Plots)
* [Summary Scalars for the night](#Summary-Scalars)
* [Jira Tickets](#Jira-Tickets)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed)
    - AuxTel
    - Simonyi
* [Exposure Log](#Exposure-Log)
    - AuxTel
    - Simonyi
* [Narrative Log](#Narrative-Log)
* [Developer Only](#dev-only)

## Night Report 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```
AuxTel had a rougher day, as the crew was delayed for the handover due to debugging new cRIO issues. We had a light hiccough with the ATAOS system faulting once, but the checkouts ran well after the team deployed another fix to correct data logging on the cRIO. We vented while waiting for the REB temperature to go down. After dinner, we attempted to take calibrations given the so-so sky quality. The ATWhite lamp CSC faulted, and waiting for the cooling cycle to complete before it could be recovered added about 30 minutes to wait time. The ATmonochromter failed next, and we were drifting into science time. We abandoned attempts of calibrations, and got on sky a bit delayed. ATPtg needed a cycle for the telescope timing issues to be sorted, but after that issue, we observed until 2 am without incident. Mostly seeing below 1.5", nearly photometric conditions. We closed at 01:45 after finishing TMA testing. Darks and biases left running.
```

Telescope Status: Telescope parked, dome and vents closed. All CSCs in STANBY.

*Authors: Alysha Shugart, Yijung Kang, Bruno Quint, Karla Pena*

### Telescope: Simonyi

```
In the morning, BLOCK-T62 was executed but just to move the TMA and left to trace the record. TMA was firstly moved to 45º. Since the default values for the EUI might be wrong, we used the EUI to set Acc and Vel manually to 0.05º/s(2, 1% performance. At 45deg and Zenith there was data collection with LaserTracker. While this process, there was an issue with a locking pin -X, which fails to move from LOCK to FREE (OBS-585). At the end, TMA was moved to Zenith (90.01 deg) and OSS/PS OFF with engaged locking pins. Note that there were interesting movements detected in M2 tangent links induced by jumping on the Az platform. 

Around 21:37pm UTC, we started BLOCK-T4, warm up camera hexapod only without issue. Warm-up M2 hexapod was done while we were waiting for BLOCK-T143 to be ready after dinner. 

We checked with Tony J. that ComCam has been already on Playback mode, BLOCK-T143 was executed after some changes updated by Tiago R. Configurations for this block were changed (see the config on OLE/Zephyr test case comment), but we had to stop at Step 2 as there was an issue with IN_PROGRESS ack for filter changes. 

M1M3 FA Bump test (BLOCK-T144) was conducted with ignoring FA423 due to its loud periodic noise. Observers went up to the level 3 to check sounds from FAs, pressure (120PSI), and temperature (deltaT ~ 1C). Actuators [133, 138, 220, 236, 240, 320, 330, 335] FAILED the bump test (SAA; ID, Pri Index: 133:32, 138:37, 220:55, 236:70, 240:74, 320:97, 330:107, 335:112, no DAA failed). 

We then started M1M3 HP breakaway test (BLOCK-145) until the end of the night. Workout Camera Hexapod (BLOCK-T87) was skipped for tonight and closed the night with BLOCK-T139, Simonyi Hexapods shutdown. MTHexapod1, MTHexapod2, and MTM1M3 were all set to DISABLED.
```

Telescope Status: TMA parked at Zenith. OSS and PS off and locking pins were engaged. MTHexapod1, MTHexapod2, and MTM1M3 were all set to DISABLED. 


*Authors: Bruno Quint, Karla Pena, David Sanmartim, Kevin Fanning, Alysha Shugart, Yijung Kang*

-------------

<a id="almanac"></a>
## Almanac

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-09-05**

,(times in UTC),
Moon Rise,2024-09-05 12:12:44.540,
Moon Set,2024-09-04 23:57:53.331,
Moon Illumination,4%,(% illuminated)
Astronomical Twilight (morning),2024-09-05 09:33:24.426,(-18 degrees)
Astronomical Twilight (evening),2024-09-04 23:49:29.974,(-18 degrees)
Solar Midnight,2024-09-05 04:41:30.776,
Nautical Twilight (morning),2024-09-05 10:01:10.597,(-12 degrees)
Nautical Twilight (evening),2024-09-04 23:21:41.930,(-12 degrees)
Civil Twilight (morning),2024-09-05 10:28:57.809,(-6 degrees)
Civil Twilight (evening),2024-09-04 22:53:52.501,(-6 degrees)


<a id="Summary-Plots"></a>
## Summary plots of whole night 

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

In [10]:
# Exposure Tally
md('<a id="Summary-Scalars"></a>')
md(f"## Summary Scalars for the night ")
md("(of all available instruments that have at least one exposure)")
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

<a id="Summary-Scalars"></a>

## Summary Scalars for the night 

(of all available instruments that have at least one exposure)

,LSSTComCamSim,LATISS
Total Night (HH:MM:SS),9:43:54,9:43:54
Total Exposure (HH:MM:SS),0:02:33,2:45:13
Number of exposures,9,286
Number of slews,40,40
Total Detector Read (HH:MM:SS),0:00:20,0:10:57
Mean Detector Read (HH:MM:SS),0:00:02,0:00:02
Total Slew (HH:MM:SS),0:41:17,0:41:17
Mean Slew (HH:MM:SS),0:01:01,0:01:01
Total Idle (HH:MM:SS),9:00:03,6:17:23


## Jira Tickets (Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [OBS-582](https://rubinobs.atlassian.net/browse/OBS-582)

- [OBS-584](https://rubinobs.atlassian.net/browse/OBS-584)

- [/rubinobs-lfa-cp/LOVE/OLE/2024/09/05/LOVE_OLE_2024-09-05T14:36:21.262.png](https://s3.cp.lsst.org/rubinobs-lfa-cp/LOVE/OLE/2024/09/05/LOVE_OLE_2024-09-05T14:36:21.262.png)

- [OBS-583](https://rubinobs.atlassian.net/browse/OBS-583)

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://summit-lsp.lsst.codes/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&min_day_obs=20240905&max_day_obs=20240907&limit=999) in `confluence_url`.

## BLOCKS observed
(TODO: BLOCKS observed) 

## Exposure Log

In [13]:
# Exposure Report
# Facet counts
if experimental.get("exposure_type_tally"):
    md("### Experimental Exposure field Tally ENABLED")
    counts = uniform_field_counts(allsrc.exp_src.exposures["LATISS"])
    for fname, tally in counts.items():
        md(f"#### Tally of values for Field: {fname}")
        display(pd.DataFrame.from_dict(tally, orient="index"))
else:
    md("### Experimental Exposure field Tally DISASBLED")

### Experimental Exposure field Tally DISASBLED

| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [14]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

-------------

## Narrative Log
(Now "linked things" from /narrativelog/messages.urls)

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-09-05 05:44:37.750878**


<pre style='color: black; background-color: LightGray; font-size: 1em'>Around 21:15 local time, shortly after signs that the network to the outside world was disconnected, ATCamera went into a fault state. The CSC reported that the MCM went into a fault state for an "unspecified reason." We stopped tracked and opened the CCS to investigate. Network issues were plaguing the summit and it was a very slow connection to the CCS-console - using shell commands saw a delay of about 1-2 seconds for every keystroke. We managed to identify the error in the ats-ih - according to the logs there was an image header delay.


*Unknown if this was due to network outages. 


We cleared the errors in the ats-ih and mcm, and we recovered the ATCamera. Even without network, we were able to resume scheduler-driven operations. The lengthy recovery time was due to the slow connection to auxtel-fp01 and reading through documentation to diagnose. 


**A note: LATISS was taking new images, but CCS was not updating, even when connected to Rubin-guest. Is this expected behavior? How might we view images if there is another network outage?</pre>


- Link: [OBS-583](https://rubinobs.atlassian.net/browse/OBS-583)

- **2024-09-05 06:01:16.348158**


<pre style='color: black; background-color: LightGray; font-size: 1em'>Logging time loss to OBS-584</pre>


...

- **2024-09-06 03:41:24.795954**


<pre style='color: black; background-color: LightGray; font-size: 1em'>Done for Block-T144. Start BLOCK-T145, hardpoint test.</pre>


- **2024-09-06 05:39:05.272652**


<pre style='color: black; background-color: LightGray; font-size: 1em'>Done for BLOCK-T145. Then run BLOCK-T139 to close the night. 
MTHexapod1, MTHexapod2, and MTM1M3 in disabled now.</pre>


-------------

<a id="dev-only"></a>
# Developer Only Section 
Contains stuff only expected to be useful to developers.

May also contain sections that have moved out of he user section because they are not defined in the Storyboard.

## Experimental 

### Parse pasted ERROR

In [16]:
import lsst.ts.logging_and_reporting.reports as rep
text = '''BEG: This is my text
that goes on for multiple lines
   and this one started with 3 spaces
                and this one with 4 tabs
back to first column.:END
'''
md(f'<code style="background-color: gray; color: yellow">{text}</code>')
md(f'<pre style="background-color: LightGray; color: black">{text}</pre>')

<code style="background-color: gray; color: yellow">BEG: This is my text
that goes on for multiple lines
   and this one started with 3 spaces
                and this one with 4 tabs
back to first column.:END
</code>

<pre style="background-color: LightGray; color: black">BEG: This is my text
that goes on for multiple lines
   and this one started with 3 spaces
                and this one with 4 tabs
back to first column.:END
</pre>

In [17]:
def htmlcode(text, fgcolor="black", bgcolor="white"):
    style = f'style="color: {fgcolor}; background-color: {bgcolor};"'
    elem = 'pre' # 'code'
    return f"<{elem} {style}>{text}</{elem}>"

md(htmlcode(text, bgcolor='LightGray'))
md(htmlcode(text, bgcolor='tomato'))

<pre style="color: black; background-color: LightGray;">BEG: This is my text
that goes on for multiple lines
   and this one started with 3 spaces
                and this one with 4 tabs
back to first column.:END
</pre>

<pre style="color: black; background-color: tomato;">BEG: This is my text
that goes on for multiple lines
   and this one started with 3 spaces
                and this one with 4 tabs
back to first column.:END
</pre>

### Widgets

In [18]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

records = [dict(a=i, b=i, c=i, d=i, e=i) for i in range(100)]


def records_toggle(show_records):
    if show_records:
        return pd.DataFrame(records)
    else:
        return print(f"Number of records: {len(records)}")


interact(records_toggle, show_records=False);

interactive(children=(Checkbox(value=False, description='show_records'), Output()), _dom_classes=('widget-inte…

## Overview 


In [19]:
try:
    import lsst.ts.logging_and_reporting.version
    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient
    have_consdb = True
except:
    have_consdb = False

In [20]:
# Display overview of Report context
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
"""
)

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://summit-lsp.lsst.codes/**
- Report **2 observing night(s)** with the last reported night starting on **2024-09-06 12:00:00**.
- min_dayobs='2024-09-05', max_dayobs='2024-09-07'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [21]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://summit-lsp.lsst.codes/rubintv-dev/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [22]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [2]

None


- Endpoint: https://summit-lsp.lsst.codes/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&min_day_obs=20240905&max_day_obs=20240907&limit=999

None
- Got 2 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [97]

None


- Endpoint: https://summit-lsp.lsst.codes/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&min_date_added=2024-09-05T00%3A00%3A00&max_date_added=2024-09-07T00%3A00%3A00&limit=999

None
- Got 97 records.  


In [23]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

## Finale

In [24]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-10-22 23:35:50.963185
